In [1]:
import pandas as pd

# Load the CSV file (make sure it's in the same folder or give the correct path)
df = pd.read_csv('drowned_all.csv')

# Normalize column names for easier use
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Preview the first few rows
print(df.head())


         date                                             platja  \
0  16/06/2024                    Platja Santa Margarida de Roses   
1  21/06/2024                     Platja de Creixell (Tarragonès   
2  24/06/2024              Platja de la Ribera de Sitges (Garraf   
3  25/06/2024                          Platja de la Gola del Ter   
4  27/06/0202  Platja de Torre Valentina a Calonge i Sant Ant...   

  municipi_o_comarca  sexe age nationality_or_undocumented  \
0        Alt Empordà  Home  65                   espanyola   
1         Tarragonès  Home  77                   espanyola   
2             Garraf  Home  55                 desconeguda   
3       Baix Empordà  Home  74                   espanyola   
4       Baix Empordà  Home  54                       russa   

      baywatch_(surveillance)         flag  morts  \
0                          sí        Groga      1   
1  Sense servei de vigilància           No      1   
2           No havia començat  Desconeguda      1   
3     

In [2]:
# Translate and clean up values
translations = {
    'sí': 'Yes',
    'no': 'No',
    'desconegut': 'Unknown',
    'desconeguda': 'Unknown',
    'verda': 'Green',
    'groga': 'Yellow',
    'vermella': 'Red',
    'no havia començat': 'Not started',
    'sense servei de vigilància': 'No surveillance'
}

# Apply translations
for col in ['baywatch_(surveillance)', 'flag']:
    df[col] = df[col].str.strip().str.lower().replace(translations).str.capitalize()

# Group by coastguard presence and flag color
coastguard_flag = df.groupby(['baywatch_(surveillance)', 'flag']).size().unstack(fill_value=0)

# Print the result
print("Drownings by Coastguard Presence and Flag Color:")
print(coastguard_flag)



Drownings by Coastguard Presence and Flag Color:
flag                             Green  No  No consta  Red  Unknown  Yellow
baywatch_(surveillance)                                                    
No                                   0   0          0    0        2       0
No consta                            0   0          1    0        0       0
No surveillance                      0  28          0    0        5       0
Not started                          0   0          0    0       15       0
Servei de vigilància finalitzat      3   1          0    2        6       2
Unknown                              4   0          0    0        8       5
Yes                                104   0          0    1        2      34


In [3]:
coastguard_flag.to_csv('drownings_flagscoastguard.csv', index=False)


In [4]:
#Top Municipalities or Comarques with Most Drownings
top_locations = df['municipi_o_comarca'].value_counts().head(10)
print("Top 10 Municipalities or Comarques:")
print(top_locations)


Top 10 Municipalities or Comarques:
municipi_o_comarca
Alt Empordà     59
Tarragonès      38
Baix Empordà    28
Garraf          22
Baix Camp       18
Baix Penedès    12
Barcelonès       7
Baix Ebre        6
Maresme          6
La Selva         5
Name: count, dtype: int64


In [5]:
top_locations.to_csv('drownings_toplocations.csv', index=False)


In [6]:
# Get top 10 municipalities or comarques
top_locations = df['municipi_o_comarca'].value_counts().head(10)

# Convert to percentage
top_locations_percentage = (top_locations / df.shape[0]) * 100

# Format and print
print("Top 10 Municipalities or Comarques (as percentages):")
print(top_locations_percentage.round(2).astype(str) + '%')


Top 10 Municipalities or Comarques (as percentages):
municipi_o_comarca
Alt Empordà     26.46%
Tarragonès      17.04%
Baix Empordà    12.56%
Garraf           9.87%
Baix Camp        8.07%
Baix Penedès     5.38%
Barcelonès       3.14%
Baix Ebre        2.69%
Maresme          2.69%
La Selva         2.24%
Name: count, dtype: object


In [7]:
top_locations_percentage.to_csv('drownings_toplocations_percentage.csv', index=False)



In [8]:
#Profile of Drowned Person

In [9]:
gender_counts = df['sexe'].value_counts()
print("Gender Distribution:")
print(gender_counts)


Gender Distribution:
sexe
Home    172
Dona     51
Name: count, dtype: int64


In [10]:
df['age'] = pd.to_numeric(df['age'], errors='coerce')  # convert age to numbers
print("Age Statistics:")
print(df['age'].describe())


Age Statistics:
count    222.000000
mean      64.175676
std       19.415223
min        4.000000
25%       55.000000
50%       70.000000
75%       78.000000
max       92.000000
Name: age, dtype: float64


In [11]:
# Replace 'desconeguda' and 'Indocumentat/da' with NaN, then count
df['nationality_or_undocumented'] = df['nationality_or_undocumented'].replace(
    {'desconeguda': None, 'Indocumentat/da': None}
)

nationality_counts = df['nationality_or_undocumented'].dropna().value_counts().head(10)
print("Top 10 Nationalities:")
print(nationality_counts)


Top 10 Nationalities:
nationality_or_undocumented
espanyola      107
francesa        32
alemanya        10
Desconeguda      7
marroquina       6
belga            5
italiana         4
romanesa         4
russa            3
anglesa          2
Name: count, dtype: int64


In [12]:
total_known = df['nationality_or_undocumented'].dropna().shape[0]  # Total count excluding NaNs
nationality_percentages = (nationality_counts / total_known) * 100  # % of total known
nationality_percentages

nationality_or_undocumented
espanyola      54.591837
francesa       16.326531
alemanya        5.102041
Desconeguda     3.571429
marroquina      3.061224
belga           2.551020
italiana        2.040816
romanesa        2.040816
russa           1.530612
anglesa         1.020408
Name: count, dtype: float64

In [13]:
nationality_percentages.to_csv('drownings_nationality_counts.csv', index=False)
